LightGBM Model

Importing Libraries

In [24]:
import lightgbm as lgb
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score


In [27]:
file_path = "../../data/processed/train_data.csv"
df = pd.read_csv(file_path)
file_path2= "../../data/processed/test_data.csv"
df2 = pd.read_csv(file_path2)

Model Functions

In [26]:

# application du binning for the features to reduce the number of splits
def apply_binning(X, num_bins=256):
    binned_data = np.zeros_like(X)
    #list to store the bin edges for each feature
    bin_edges_list = []
    
    for i in range(X.shape[1]):
        col = X[:, i]
        #min and max values of the feature for sorting the values before binning
        bins = np.linspace(col.min(), col.max(), num_bins + 1)
        #digitize function to bin the values and subtract 1 to start the bin index from 0
        binned_data[:, i] = np.digitize(col, bins) - 1
        bin_edges_list.append(bins)
    
    return binned_data, bin_edges_list

# function to initialize the predictions with the log_odds for avoiding the overflow with the values of 0 and 1
def initialize_predictions(y):
    p = np.mean(y)
    log_odds = np.log(p / (1 - p))
    return np.full_like(y, 1 / (1 + np.exp(-log_odds)), dtype=np.float32)

# function to compute the residuals and hessians for the gradient boosting
def compute_residuals_and_hessians(y_true, y_pred):
    residuals = y_pred - y_true
    #hessians for calculating the second order derivative for knowing the curvature of the loss function
    hessians = y_pred * (1 - y_pred)
    return residuals, hessians

# function to update the predictions after finding the best split
def update_predictions(X, best_feature, best_split, gradients, hessians, learning_rate):
    predictions_update = np.zeros_like(gradients)
    #mask for the samples in the left node
    
    left_mask = X[:, best_feature] < best_split
    #mask for the samples in the right node
    right_mask = ~left_mask
    #calculating the gradients and hessians for the left and right nodes
    left_grad = gradients[left_mask].sum()
    left_hess = hessians[left_mask].sum()
    right_grad = gradients[right_mask].sum()
    right_hess = hessians[right_mask].sum()
    
    if left_hess > 0:
        left_value = -left_grad / left_hess
    else:
        left_value = 0
    
    if right_hess > 0:
        right_value = -right_grad / right_hess
    else:
        right_value = 0
    
    predictions_update[left_mask] = left_value
    predictions_update[right_mask] = right_value
    
    return predictions_update * learning_rate

def find_best_split(X, gradients, hessians, num_bins):
    # Initialize with negative infinity for assurring taht the first gain will be higher
    best_gain = -np.inf
    best_feature, best_split = None, None

    for feature in range(X.shape[1]): 
        bin_gradients = np.zeros(num_bins)
        bin_hessians = np.zeros(num_bins)
        #calculating the gradients and hessians for each bin 
        for bin_idx in range(num_bins):
            #mask for the samples in the bin 
            mask = (X[:, feature] == bin_idx)  
            #summing the gradients and hessians for the samples in the bin for assruing that the bin is not empty
            bin_gradients[bin_idx] = gradients[mask].sum()
            bin_hessians[bin_idx] = hessians[mask].sum()
        #searching for the best split
        for split in range(1, num_bins):
            #calculating the gain for the split for left and right nodes because the gain is the sum of the gains of the left and right nodes
            left_grad = bin_gradients[:split].sum()
            left_hess = bin_hessians[:split].sum()
            right_grad = bin_gradients[split:].sum()
            right_hess = bin_hessians[split:].sum()
            #if the hessians are positive we can calculate the gain
            if left_hess > 0 and right_hess > 0:
                gain = (left_grad ** 2 / left_hess) + (right_grad ** 2 / right_hess)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_split = split

    return best_feature, best_split, best_gain
# function to train the lightgbm model
def train_lightgbm(X, y, num_bins=64, learning_rate=1.2, num_trees=10):
    X_binned, bin_edges = apply_binning(X, num_bins=num_bins)
    
    initial_prediction = np.log(y.mean() / (1 - y.mean()))
    y_pred = np.full_like(y, initial_prediction, dtype=float)
    
    trees = []
    
    for tree in range(num_trees):
        gradients, hessians = compute_residuals_and_hessians(y, 1 / (1 + np.exp(-y_pred)))
        
        best_feature, best_split, best_gain = find_best_split(X_binned, gradients, hessians, num_bins)
        predictions_update = update_predictions(X_binned, best_feature, best_split, gradients, hessians, learning_rate)
        y_pred += predictions_update
        
        tree_structure = {
            'feature': best_feature,
            'split': best_split,
            'left_value': -gradients[X_binned[:, best_feature] < best_split].sum() / hessians[X_binned[:, best_feature] < best_split].sum(),
            'right_value': -gradients[X_binned[:, best_feature] >= best_split].sum() / hessians[X_binned[:, best_feature] >= best_split].sum(),
        }
        trees.append(tree_structure)
    
    return trees, initial_prediction


def predict_lightgbm(X, trees, initial_prediction, num_bins=64):
    X_binned, _ = apply_binning(X, num_bins=num_bins)
    y_pred = np.full(X.shape[0], initial_prediction, dtype=float)
    
    for tree in trees:
        feature = tree['feature']
        split = tree['split']
        left_value = tree['left_value']
        right_value = tree['right_value']
        
        mask = X_binned[:, feature] < split
        y_pred[mask] += left_value
        y_pred[~mask] += right_value
    
    return 1 / (1 + np.exp(-y_pred))


Implemented the lightgbm model with the training and prediction functions and applied it to the train and test data

Training model with  all features 

In [30]:

X_train= df.drop(columns=['target']).values
y_train= df['target'].values
X_test= df2.drop(columns=['target']).values
y_test= df2['target'].values
num_bins = 256
learning_rate = 0.1
num_trees = 100

trained_trees, initial_prediction = train_lightgbm(X_train, y_train, num_bins=num_bins, learning_rate=learning_rate, num_trees=num_trees)

y_pred_test = predict_lightgbm(X_test, trained_trees, initial_prediction, num_bins=num_bins)

loss = log_loss(y_test, y_pred_test)
print(f"Log Loss sur le jeu de test : {loss}")
accuracy = accuracy_score(y_test, (y_pred_test >= 0.5).astype(int))

print(f"Accuracy sur le jeu de test : {accuracy * 100:.2f}%")

Log Loss sur le jeu de test : 1.0564933517785018
Accuracy sur le jeu de test : 82.27%
